# Always run this code on the first day of the month!

# Import

In [1]:
import re
import requests
import pandas        as pd

from bs4             import BeautifulSoup
from datetime        import datetime

# API building

In [2]:
#Link do acesso
url = requests.get("https://www.autoscout24.com/lst/land-rover?atype=C&desc=0&sort=standard&source=homepage_search-mask&ustate=N%2CU")

#Salvando o objeto
content = url.content
url_final = []

#Convertendo para um objeto do tipo BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

list_auto = []

# Acessando todos os produtos

In [3]:
def gerar_links_base(prefixo, sufixo, num_iteracoes, final_url):
    links = []

    for i in range(1, num_iteracoes + 1):
        link = f"{prefixo}{i}{sufixo}{final_url}"
        links.append(link)

    return links

# construindo o link:
prefixo = "https://www.autoscout24.com/lst/land-rover/defender?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_listing=exclude&desc=0&page="
sufixo = ""
num_iteracoes = 20
final_url = "&powertype=kw&search_id=2gn5j092pnr&sort=standard&source=listpage_pagination&ustate=N%2CU"
            

links_gerados = gerar_links_base(prefixo, sufixo, num_iteracoes, final_url)
print(links_gerados)

['https://www.autoscout24.com/lst/land-rover/defender?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_listing=exclude&desc=0&page=1&powertype=kw&search_id=2gn5j092pnr&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/land-rover/defender?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_listing=exclude&desc=0&page=2&powertype=kw&search_id=2gn5j092pnr&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/land-rover/defender?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_listing=exclude&desc=0&page=3&powertype=kw&search_id=2gn5j092pnr&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/land-rover/defender?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_listing=exclude&desc=0&page=4&powertype=kw&search_id=2gn5j092pnr&sort=standard&source=listpage_pagination&ustate=N%2CU', 'https://www.autoscout24.com/lst/land-rover/defender?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_

## Acessando os produtos e construindo a tabela


In [5]:
# Teste dois
for link in links_gerados:
    #print(link)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get( link )

    #Salvando o objeto
    content = page.content
    #url_final = []
    
    #Convertendo para um objeto do tipo BeautifulSoup
    soup = BeautifulSoup(content, 'html.parser')

    # Beautiful Soup object
    soup = BeautifulSoup( page.text, 'html.parser' )

    add = soup.find_all('article', re.compile('cldt-summary-full-item'))
     #Seller/Location
    seller_detail = soup.find('div', attrs={'class':re.compile('SellerInfo')})
    seller = seller_detail.find('span', attrs={'class':re.compile('SellerInfo_name')}).get_text().strip()
    endereco = seller_detail.find('span', attrs={'class':re.compile('SellerInfo_address')}).get_text().strip()
        
      
    for ads in add:
        #auto = soup.find('a', attrs={'class':re.compile('ListItem_title')}).get_text().strip()
        id_ads = ads['id']
        manufacturer_by = ads['data-make']
        model = ads['data-model']
        km = ads['data-mileage']
        year = ads['data-first-registration']
        price = ads['data-price']
        version = ads.find('span', attrs={'class': re.compile("ListItem_version")}).getText()
        description_car = ads.find('div', attrs={'class': re.compile("ListItem_wrapper")}).getText()
        
        list_auto.append([id_ads, manufacturer_by, model, version, km, year, price, seller, endereco])
        
        df = pd.DataFrame(list_auto, columns=['id_ads', 'manufacturer_by', 'model', 'version',
                                              'km', 'year', 'price', 'seller',
                                              'endereco']).reset_index(drop=True)
        df['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        

        

## Armazenando os dados

→ Ao rodar este codeigo no inicio do mês, a tabela original será substituida. Para que isso não ocorra, deve-se alterar o nome do arquivo atual para outro.

In [6]:
rows = df.shape[0]
print(rows)

# Salve a tabela concatenada no novo arquivo
df.to_csv('../dataset/list_auto.csv', encoding='utf-8', sep=';' ,index=False)


395
